In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.in_out.basics import read_saved_epochs          
from tf_lab.data_sets.shape_net import snc_category_to_synth_id, load_all_point_clouds_under_folder

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_points = 2048
do_training = True
first_time_running = True
load_pretrained_model = False
load_epoch = None
seed = 42

In [5]:
max_training_epochs = 1000
loss = 'EMD'
bneck_size = 64

class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]

experiment_id = raw_input('Give me the experiment ID:').lower()

experiment_name = '_'.join([class_name, experiment_id, str(n_pc_points), 
                           'pts_bneck', str(bneck_size), loss
                           ])

top_data_dir = '/orions4-zfs/projects/optas/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings', experiment_name)
create_dir(train_dir)

Give me the class type: chair
Give me the experiment ID:contrasting_neighborhoods


'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_contrasting_neighborhoods_2048_pts_bneck_64_EMD'

In [6]:
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))
pclouds_path = osp.join(pclouds_path, syn_id)
train_data = load_all_point_clouds_under_folder(pclouds_path, n_threads=40, verbose=True)

6778 pclouds were loaded. They belong in 1 shape-classes.


In [22]:
from general_tools.rla.three_d_transforms import rand_rotation_matrix

r_rotation = rand_rotation_matrix()
r_rotation[0, 2] = 0
r_rotation[2, 0] = 0
r_rotation[1, 2] = 0
r_rotation[2, 1] = 0
r_rotation[2, 2] = 1

Point_Cloud(train_data.point_clouds[0].dot(r_rotation)).lex_sort()[1]



array([ 214,  209,  194, ..., 2044, 2043, 2042])

In [122]:
encoder_args = { 'n_filters': [64, 128, 256, 512],
                 'filter_sizes': [8, 6, 4, 4],
                 'strides': [1, 1, 2, 2],
                 'b_norm': False,               
                }

decoder_args = {'layer_sizes': [1024, 2048, n_pc_points*3], 'b_norm': False}

In [130]:
if load_pretrained_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    if conf.train_dir != train_dir:
        conf.train_dir = train_dir
    conf.save(osp.join(train_dir, 'configuration'))
else:
    conf = Conf(
                n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = 1000,
                batch_size = 50,
                denoising = False,
                learning_rate = 0.0005,
                train_dir = train_dir,              
                loss_display_step = 1,
                saver_step = 20,
                z_rotate = False,
                encoder = encoder_with_convs_and_symmetry,
                decoder = decoder_with_fc_only,
                encoder_args = encoder_args,
                decoder_args = decoder_args
               )    
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [131]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

In [111]:
if load_pretrained_model:
    if load_epoch is None: # load last saved.
        saved_epochs = read_saved_epochs(conf.train_dir)
        load_epoch = saved_epochs[-1]        
    ae.restore_model(conf.train_dir, load_epoch, verbose=True)

In [127]:
p = Point_Cloud(train_data.point_clouds[1])
_, ind = p.lex_sort()

In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.1888', 'loss=', '0.003647510')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_for_contrasting_neighbhors_2048_pts_bneck_64_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_for_contrasting_neighbhors_2048_pts_bneck_64_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0002', 'training time (minutes)=', '0.1906', 'loss=', '0.002043176')
('Epoch:', '0003', 'training time (minutes)=', '0.1818', 'loss=', '0.001700465')
('Epoch:', '0004', 'training time (minutes)=', '0.1915', 'loss=', '0.001467570')